In [2]:
from langchain import hub
from langchain.agents import (
    AgentExecutor,
    create_openai_functions_agent,
    create_openai_tools_agent,
) # you can use either functions agent or tools agent - we recommend using tools agent as functions are deprecated 
from langchain_openai import ChatOpenAI


In [3]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [62]:
open_source_model = ChatOpenAI(
    base_url=os.environ['MYGEN_API'],
    model="gpt-35-turbo",
    # model="deepmind/meta-llama-3-70b-instruct", 
    api_key=os.environ['MYGEN_KEY'],
    streaming=False,
)

In [54]:
open_source_model.invoke("What is Collibra data catalog?")

AIMessage(content="Collibra is a data catalog platform that helps organizations manage and make sense of their data assets. It provides a centralized inventory of an organization's data, making it easier to find, understand, and use data to drive business decisions.\n\nCollibra's data catalog is designed to help organizations overcome common data challenges, such as data sprawl, data quality issues, and lack of data visibility. The platform provides a single source of truth for an organization's data, allowing users to discover, understand, and trust their data.\n\nKey features of Collibra's data catalog include:\n\n1. Data Discovery: Collibra's data catalog allows users to search and discover data across the organization, regardless of where it's stored.\n2. Data Profiling: The platform provides detailed information about each data asset, including its structure, quality, and relationships with other data assets.\n3. Data Lineage: Collibra's data catalog tracks the origin, movement, a

In [37]:
import numexpr
from langchain import LLMMathChain
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor, Tool
from langchain_core.prompts import ChatPromptTemplate


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("placeholder", "{chat_history}"),  # if you want to include the chat history
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"), # intermediate agent actions and tool output messages will be passed in here
    ]
)


llm_math_chain = LLMMathChain.from_llm(llm=open_source_model, verbose=True)

tools = [Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    )]

agent = create_openai_functions_agent(open_source_model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [47]:
from langchain_community.tools import DuckDuckGoSearchResults
search = DuckDuckGoSearchResults(num_results=1)
search.run("What is collibra?")

'[snippet: Collibra is a leading data governance and data catalog software platform that helps organizations manage, govern, and catalog their data assets effectively. It provides a centralized and collaborative environment for data governance, data lineage, data stewardship, and data cataloging. Collibra enables data-driven organizations to ensure data ..., title: What is Collibra and use cases of Collibra? - DevOpsSchool.com, link: https://www.devopsschool.com/blog/what-is-collibra-and-use-cases-of-collibra/]'

In [36]:
agent_executor.invoke({"input": "what is 3096 divided by 3439.6?"})



> Entering new AgentExecutor chain...

Invoking: `Calculator` with `3096 / 3439.6`




> Entering new LLMMathChain chain...
3096 / 3439.6```text
3096 / 3439.6
```
...numexpr.evaluate("3096 / 3439.6")...

Answer: 0.9001046633329457
> Finished chain.
Answer: 0.90010466333294573096 divided by 3439.6 is approximately 0.9001

> Finished chain.


{'input': 'what is 3096 divided by 3439.6?',
 'output': '3096 divided by 3439.6 is approximately 0.9001'}

In [63]:
from langchain_community.tools import DuckDuckGoSearchResults

duckduck_tool = DuckDuckGoSearchResults(
    name="duck_duck_go_search",
    verbose=True,
    num_results=1
)

tools = [
    duckduck_tool
]

In [64]:
prompt = hub.pull("hwchase17/openai-tools-agent")

agent = create_openai_tools_agent(
    open_source_model,
    tools,
    prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [65]:
response = agent_executor.invoke(
    {
        "input": "use tools agent to find information about collibra data catalog",
    }
)



> Entering new AgentExecutor chain...

Invoking: `duck_duck_go_search` with `{'query': 'Collibra Data Catalog'}`


[snippet: Collibra's proven, pre-configured services get your integrations, data catalog, and lineage ready to go in days instead of months. The implementation is designed to deliver the most important, high-impact features first so that your team can start extracting immediate value. Avoid risks with established best practices that ensure things get ..., title: Getting started with a data catalog | Collibra, link: https://www.collibra.com/us/en/blog/getting-started-with-a-data-catalog]Collibra is known for its proven, pre-configured services that get integrations, data catalog, and lineage ready to go in days instead of months. The implementation is designed to deliver the most important, high-impact features first so that your team can start extracting immediate value. You can find more information about getting started with a data catalog from Collibra on their websit

In [66]:
response

{'input': 'use tools agent to find information about collibra data catalog',
 'output': 'Collibra is known for its proven, pre-configured services that get integrations, data catalog, and lineage ready to go in days instead of months. The implementation is designed to deliver the most important, high-impact features first so that your team can start extracting immediate value. You can find more information about getting started with a data catalog from Collibra on their website: [Getting started with a data catalog | Collibra](https://www.collibra.com/us/en/blog/getting-started-with-a-data-catalog)'}